In [ ]:
input_dir = "input"

In [ ]:
# add project root to module path if not added already
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    print(f"Path '{module_path}' added to module path.")


In [ ]:
from tools.csv_consolidator import consolidate

csv_inputs = consolidate(input_dir)

In [ ]:
import pandas

df = pandas.DataFrame.from_records([m.dict() for m in csv_inputs]).drop_duplicates()

### Profit calculation

In [ ]:
buy_df = df \
    .query("action == 'buy' | action == 'buy limit'") \
    .assign(invested = lambda x: x.total_eur - x.french_tax - x.conversion_fee_eur - x.stamp_duty_tax_eur) \
    .groupby(["ticker", "name"]) \
    .agg(invested = ("invested", "sum"), shares_bought = ("shares_count", "sum"))

# display(buy_df)

In [ ]:
sell_df = df \
    .query("action == 'sell' | action == 'sell limit'") \
    .assign(sell_gain = lambda x: x.total_eur - x.french_tax - x.conversion_fee_eur - x.stamp_duty_tax_eur) \
    .groupby(["ticker", "name"]) \
    .agg(sell_gain = ("sell_gain", "sum"), shares_sold = ("shares_count", "sum"))

# display(sell_df)

In [ ]:
import numpy as np

def get_shares_remaining(x):
    d = x.shares_bought - x.shares_sold
    return np.where(abs(d) > 0.1, d, 0)


def zero_if_none(column):
    return np.where(column != "NaN", column, 0)



profit_df = buy_df.join(sell_df, on=["ticker", "name"], how="outer") \
    .fillna(0) \
    .assign(
        profit = lambda x: x.sell_gain - x.invested,
        shares_remaining = lambda x: get_shares_remaining(x),
    ) \
    .assign(profit_yield = lambda x: x.invested / x.profit) \
    .query("shares_remaining == 0 | shares_remaining < shares_bought")[["profit", "profit_yield", "shares_remaining"]]


display(profit_df)

In [ ]:
total_profit_df = profit_df.agg(all_profits = ("profit", "sum"))

display(total_profit_df)

### Dividend calculation

In [ ]:
dividend_df = df \
    .query("action == 'dividend'") \
    .groupby(["ticker", "name"]) \
    .agg(dividend = ("total_eur", "sum"), withholding_tax = ("withholding_tax", "sum"), tax_currency = ("withholding_tax_currency", "first"))


display(dividend_df)

In [ ]:
total_dividend_df = dividend_df.agg(all_dividends = ("dividend", "sum"))

display(total_dividend_df)

### Total gains

In [ ]:
profits = total_profit_df.to_dict()['profit']['all_profits']
dividends = total_dividend_df.to_dict()['dividend']['all_dividends']

print('Total gains: {:.2f}'.format(profits + dividends))